<a href="https://colab.research.google.com/github/Straiyanz/PHYS434/blob/main/Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 7/8: HERA
## Lab Partner: Oscar Pujol
---
### Step 1: Remove Contamination

In [1]:
# so i dont have to use a terminal every time :), thanks oscar
!pip install pyuvdata > /dev/null # /dev/null just supresses output

In [2]:
# imports
from pyuvdata import UVData
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from scipy.stats import rayleigh
import gc
import ctypes

In [3]:
# loading data
path_to_data_file = './HERA/'

uv1 = UVData() # naming after last digit in file numbering scheme, so 1,5,8 respectively
uv1.read(f'{path_to_data_file}/zen.2459122.34011.mini.sum.uvh5')
data1 = uv1.data_array

# uv5 = UVData()
# uv5.read(f'{path_to_data_file}/zen.2459122.48015.mini.sum.uvh5')
# data5 = uv5.data_array

# uv8 = UVData()
# uv8.read(f'{path_to_data_file}/zen.2459122.62018.mini.sum.uvh5')
# data8 = uv8.data_array

# doing this to refresh my memory on the sizes of things
print('This data array has shape:')
print('(# baselines*times, # spectral windows, # frequencies, # polarizations)')
print(np.shape(data1))

FileNotFoundError: ignored

In [ ]:
# selecting antenna with data:
ants = uv1.get_ants()
print(ants)

In [ ]:
# getting baseline data for each
alldata = []
for i in range(len(ants)):
    for j in range(len(ants)):
        newdata = np.abs(uv1.get_data(ants[i], ants[j]) - uv1.get_data(ants[i], ants[i]))
        if newdata.all() != 0.0:
            alldata.append(newdata)

In [ ]:
# makes upcoming for loops easier
baselines = range(len(alldata))
print(baselines)
t = range(len(alldata[0]))
print(t)
freqs = range(len(alldata[0][0]))
print(freqs)

# making space kernel keeps crashing
del data1, uv1

In [ ]:
alldata_after = np.copy(alldata)
# looping over all baselines with data
for k in baselines:
    # creating masks
    mask = np.ones((20, len(alldata[0][0])))
    mask_plot = np.zeros_like(mask)
    # looping over time intervals
    for i in range(19):
        data_i = (alldata[k][i])
        data_f = (alldata[k][i+1])
        data = np.sqrt((data_f - data_i)**2)
        threshhold = np.mean(data)
        # looping over frequencies
        for j in freqs:
            if data[j] >= threshhold:
                mask[i][j] = 0 # setting mask to zero where we want cuts to be made
                mask_plot[i][j] = data[j] # this is purely for plotting

        # plotting to check myself
        if k % round(len(alldata)/2) == 0 and i % 20 == 0:
            plt.figure(figsize=(12,6))
            plt.plot(data)
            plt.plot(mask_plot[i], color='green', label='mask')
            plt.plot([0, len(data)],[threshhold, threshhold], color='red', label='cut')
            plt.title(f'Baseline {k}, Timeslice Comparison: {i} to {i+1}')
            plt.xlabel('Frequency (MHz)')
            plt.ylabel('Amplitude')
            plt.yscale('log')
            plt.legend()
            plt.show()

    mask_u = np.ones(len(alldata[0][0])) #  creating mask union
    for i in range(20):
        mask_u *= mask[i]
        alldata_after[k][i] *= mask_u # multiplying data by this sets data to zero where we want to cut


del mask_u, mask

In [ ]:
# checking time slice 0 for a couple baselines to check
xlab = 'Frequency (MHz)'
ylab = 'Amplitude'

for k in baselines:
    if k % round(len(alldata)/2) == 0:
        plt.figure(figsize=(12,6))
        plt.plot(alldata[k][0])
        plt.title(f'Baseline {k}, Time Slice 0 - Before Filtering')
        plt.xlabel(xlab)
        plt.ylabel(ylab)
        plt.show()

        plt.figure(figsize=(12,6))
        plt.plot(alldata_after[k][0])
        plt.title(f'Baseline {k}, Time Slice 0 - After Filtering')
        plt.xlabel(xlab)
        plt.ylabel(ylab)
        plt.show()

### Step 2: Investigating |$\Delta V$|

In [ ]:
# flattening to create histogram
dv = np.copy(alldata_after).flatten() # was killing my kernal so want to calculate this at least as possible

In [ ]:
# putting frequencies into histogram
# dv1 = np.copy(alldata_after[0]).flatten()

# histogram
plt.figure(figsize=(12,6))
bins = plt.hist(dv, bins=100, density=False, label='dV distribution')
plt.axvline(x=0.9e7, color='red', linestyle='dashed', label='selection')
plt.axvline(x=1e7, color='red', linestyle='dashed')



# prettifying
plt.title('|V| Histogram')
plt.ylabel('Count')
plt.xlabel('|V|')
plt.xlim(-0.1e6, 25.1e6)
plt.show()

In [ ]:
libc = ctypes.CDLL("libc.so.6") # clearing cache
libc.malloc_trim(0)
cut_u = 1e7
cut_l = 0.9e7
dv1 = []
for k in tqdm(baselines):
    for i in t:
        for j in freqs:

            if alldata_after[k][i][j] > cut_l and alldata_after[k][i][j] < cut_u:
                dv1.append(alldata[k][i][j])


In [ ]:
gc.collect()
plt.hist(dv1, bins=30)
plt.show()